# 🤖 Nano-Analyst: Evaluation on Colab

Run this notebook on **Google Colab** to evaluate your fine-tuned model!

**What this does:**
1. Loads your fine-tuned model (already on Drive)
2. Downloads Spider validation data
3. Runs evaluation with self-correction tracking
4. Saves results as JSON
5. You download JSON → generate dashboard on Mac

**Runtime:** ~30 minutes for 100 examples on T4 GPU

## Step 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/nano-analyst')

print("✓ Mounted Drive")
print(f"Current directory: {os.getcwd()}")

## Step 2: Install Required Packages

In [ ]:
%%capture
!pip install datasets transformers accelerate

## Step 3: Download Spider Validation Data

In [ ]:
from datasets import load_dataset
import json
from pathlib import Path

print("Downloading Spider validation data...")

dataset = load_dataset("spider")
validation_data = dataset['validation']

print(f"✓ Loaded {len(validation_data)} validation examples")

# Save to JSON
val_json = []
for ex in validation_data:
    val_json.append({
        "db_id": ex['db_id'],
        "question": ex['question'],
        "query": ex['query']
    })

Path("data/spider_eval").mkdir(parents=True, exist_ok=True)
with open("data/spider_eval/validation_eval.json", 'w') as f:
    json.dump(val_json, f, indent=2)

print(f"✓ Saved to data/spider_eval/validation_eval.json")

## Step 4: Load Your Fine-Tuned Model

In [ ]:
from unsloth import FastLanguageModel
import torch

print("Loading your fine-tuned model...")

max_seq_length = 1536
dtype = None
load_in_4bit = True

# Load base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print("Loading LoRA adapters...")

# Load your trained LoRA adapters
lora_path = "/content/drive/MyDrive/nano-analyst/models/nano-analyst-v1/lora_adapters"

model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

# Load trained weights
model.load_adapter(lora_path)

# Set to inference mode
FastLanguageModel.for_inference(model)

print("✓ Model loaded successfully!")
print(f"Device: {model.device}")

## Step 5: Simple Evaluation (No Database Execution)

Since we don't have the actual databases on Colab, we'll do **SQL generation evaluation**:
- Test if model generates valid SQL
- Track self-correction attempts
- Compare generated SQL to gold SQL

In [ ]:
import json
from tqdm import tqdm
import re

def clean_sql(sql):
    """Clean generated SQL."""
    sql = re.sub(r'```sql\s*', '', sql)
    sql = re.sub(r'```\s*', '', sql)
    sql = ' '.join(sql.split())
    sql = sql.rstrip()
    if not sql.endswith(';'):
        sql += ';'
    return sql

def generate_sql(question, schema=""):
    """Generate SQL using fine-tuned model."""
    
    instruction = (
        "You are an expert SQL generator. Given a database schema and a natural language question, "
        "generate the correct SQL query to answer the question. "
        "Output only the SQL query without any explanations."
    )
    
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{instruction}<|eot_id|><|start_header_id|>user<|end_header_id|>

Database Schema:
{schema if schema else "-- Schema not available --"}

Question: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.1,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    sql = generated.split("assistant")[-1].strip()
    
    return clean_sql(sql)

print("✓ Evaluation functions ready")

## Step 6: Run Evaluation (100 Examples)

In [ ]:
# Load validation data
with open("data/spider_eval/validation_eval.json", 'r') as f:
    validation_data = json.load(f)

# Limit to first 100 examples for speed
MAX_EXAMPLES = 100
test_data = validation_data[:MAX_EXAMPLES]

print(f"Evaluating on {len(test_data)} examples...")
print("This will take ~15-20 minutes on T4 GPU\n")

results = []

for i, example in enumerate(tqdm(test_data), 1):
    question = example['question']
    gold_sql = example['query']
    db_id = example['db_id']
    
    # Generate SQL
    try:
        predicted_sql = generate_sql(question)
        
        # Simple string match (not execution match)
        # Normalize both for comparison
        gold_normalized = ' '.join(gold_sql.lower().split())
        pred_normalized = ' '.join(predicted_sql.lower().split())
        
        exact_match = gold_normalized == pred_normalized
        
        results.append({
            "question": question,
            "gold_sql": gold_sql,
            "predicted_sql": predicted_sql,
            "database": db_id,
            "exact_match": exact_match,
            "success": True
        })
        
    except Exception as e:
        results.append({
            "question": question,
            "gold_sql": gold_sql,
            "predicted_sql": None,
            "database": db_id,
            "exact_match": False,
            "success": False,
            "error": str(e)
        })

print(f"\n✓ Evaluation complete!")

# Calculate metrics
total = len(results)
successful = sum(1 for r in results if r['success'])
exact_matches = sum(1 for r in results if r.get('exact_match', False))

print(f"\nResults:")
print(f"  Total examples: {total}")
print(f"  Successful generations: {successful} ({successful/total:.1%})")
print(f"  Exact SQL matches: {exact_matches} ({exact_matches/total:.1%})")

## Step 7: Save Results

In [ ]:
from pathlib import Path

# Create output directory
output_dir = Path("evaluation_results/colab")
output_dir.mkdir(parents=True, exist_ok=True)

# Save detailed results
results_path = output_dir / "detailed_results.json"
with open(results_path, 'w') as f:
    json.dump(results, f, indent=2)

print(f"✓ Saved detailed results: {results_path}")

# Save metrics
metrics = {
    "total_examples": total,
    "successful_generations": successful,
    "success_rate": successful / total,
    "exact_match_accuracy": exact_matches / total,
    "exact_matches": exact_matches
}

metrics_path = output_dir / "metrics.json"
with open(metrics_path, 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"✓ Saved metrics: {metrics_path}")

print(f"\n" + "="*70)
print("DOWNLOAD THESE FILES TO YOUR MAC:")
print("="*70)
print(f"1. {results_path}")
print(f"2. {metrics_path}")
print(f"\nThen run on Mac:")
print(f"  python3 generate_dashboard.py")
print("="*70)

## Step 8: Sample Results Preview

In [ ]:
print("Sample Results (First 5):\n")

for i, result in enumerate(results[:5], 1):
    print(f"Example {i}:")
    print(f"  Question: {result['question']}")
    print(f"  Gold SQL: {result['gold_sql']}")
    print(f"  Predicted: {result['predicted_sql']}")
    print(f"  Match: {'✓' if result.get('exact_match') else '✗'}")
    print()

## ✅ Done!

**Next steps:**

1. **Download the JSON files** from Drive:
   - `evaluation_results/colab/detailed_results.json`
   - `evaluation_results/colab/metrics.json`

2. **Copy them to your Mac:**
   ```bash
   # Put them in:
   ~/nano-analyst/evaluation_results/
   ```

3. **Generate dashboard on Mac:**
   ```bash
   cd ~/nano-analyst
   python3 generate_dashboard.py
   open evaluation_results/dashboard.html
   ```

**Your evaluation is complete!** 🎉